In [1]:
pip install binance-futures-connector

   ---------------------------------------- 0.0/58.8 kB ? eta -:--:--
   ---------------------------------------- 58.8/58.8 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 0.58.0
    Uninstalling websocket-client-0.58.0:
      Successfully uninstalled websocket-client-0.58.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install ta

Note: you may need to restart the kernel to use updated packages.


In [62]:
import os
from dotenv import load_dotenv

load_dotenv('majorkeys.env')

api_key = os.getenv('API_KEY')
api_secret = os.getenv('API_SECRET')


In [63]:
from binance.um_futures import UMFutures
import ta
import pandas as pd
from time import sleep
from binance.error import ClientError

client = UMFutures(api_key, api_secret)

In [64]:
# 0.012 means +1.2%, 0.009 is -0.9%
tp = 0.012
sl = 0.009
volume = 10  # Amount of USDT to use for each trade, if leverage 10 then you invest 1 USDT approx., needs to be big enough
leverage = 10
type = 'ISOLATED'  # type of margin is 'ISOLATED' or 'CROSS'
qty = 100  # Amount of concurrent opened positions

# getting your binance balance in USDT
def get_balance_usdt():
    try:
        response = client.balance(recvWindow=6000)
        for elem in response:
            if elem['asset'] == 'USDT':
                return float(elem['balance'])

    except ClientError as error:
        print(
            "Found error. status: {}, error code: {}, error message: {}".format(
                error.status_code, error.error_code, error.error_message
            )
        )

get_balance_usdt()


10000000000000000000000000000000000000000000

In [65]:
# get all USDT pairs 
def get_tickers_usdt():
    tickers = []
    resp = client.ticker_price()
    for elem in resp:
        if 'USDT' in elem['symbol']:
            tickers.append(elem['symbol'])
    return tickers

get_tickers_usdt()

['POLYXUSDT',
 'LUNA2USDT',
 'MANAUSDT',
 'CVXUSDT',
 'ETHWUSDT',
 '1000BONKUSDT',
 'FRONTUSDT',
 'EOSUSDT',
 'ONEUSDT',
 'EGLDUSDT',
 'MASKUSDT',
 'IDEXUSDT',
 'GMTUSDT',
 'GRTUSDT',
 'DARUSDT',
 'XMRUSDT',
 'CRVUSDT',
 'INJUSDT',
 'RONINUSDT',
 'MAVUSDT',
 'JUPUSDT',
 'GASUSDT',
 'LEVERUSDT',
 'XRPUSDT',
 'ETHUSDT_240927',
 'AEVOUSDT',
 'GALAUSDT',
 'AUCTIONUSDT',
 'RIFUSDT',
 'MTLUSDT',
 'XVGUSDT',
 'ZETAUSDT',
 'UNFIUSDT',
 'SNTUSDT',
 'LINAUSDT',
 'NEARUSDT',
 'UMAUSDT',
 'ORDIUSDT',
 'ARPAUSDT',
 'WIFUSDT',
 'BLUEBIRDUSDT',
 'ALPHAUSDT',
 'DUSKUSDT',
 'SANDUSDT',
 'FETUSDT',
 'ROSEUSDT',
 'RUNEUSDT',
 'STGUSDT',
 'MBLUSDT',
 'QNTUSDT',
 'APEUSDT',
 'RLCUSDT',
 'HOOKUSDT',
 'BLZUSDT',
 'LINKUSDT',
 'TOKENUSDT',
 'ARKUSDT',
 'JTOUSDT',
 'DODOXUSDT',
 'IOTXUSDT',
 'OMUSDT',
 'ZRXUSDT',
 'PIXELUSDT',
 'VANRYUSDT',
 'REZUSDT',
 'NOTUSDT',
 'FLMUSDT',
 'THETAUSDT',
 'IMXUSDT',
 'ARBUSDT',
 'KEYUSDT',
 'SPELLUSDT',
 'LITUSDT',
 'SUIUSDT',
 'TONUSDT',
 'ILVUSDT',
 'DASHUSDT',
 'WUSDT',
 

In [66]:
#get information of a pair, last 500 instances
def klines(symbol):
    try:
        resp = pd.DataFrame(client.klines(symbol, '1m'))
        resp = resp.iloc[:,:6] # only get the first six columns and rename them:
        resp.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
        resp = resp.set_index('Time') # replace index with time
        resp.index = pd.to_datetime(resp.index, unit = 'ms') # reformat time value
        resp = resp.astype(float) #float so we can use them
        return resp
    except ClientError as error:
        print(
            "Found error. status: {}, error code: {}, error message: {}".format(
                error.status_code, error.error_code, error.error_message
            )
        )
        
klines('BTCUSDT')

,Open,High,Low,Close,Volume
Time,,,,,
2024-05-25 06:19:00,68715.1,68715.1,68703.0,68703.0,54.748
2024-05-25 06:20:00,68703.0,68717.8,68689.3,68717.8,88.197
2024-05-25 06:21:00,68717.7,68717.8,68703.5,68703.5,17.202
2024-05-25 06:22:00,68703.5,68703.6,68699.1,68699.1,19.050
2024-05-25 06:23:00,68699.2,68716.9,68699.1,68716.9,39.368
...,...,...,...,...,...
2024-05-25 14:34:00,69053.0,69063.7,69050.4,69054.7,45.426
2024-05-25 14:35:00,69054.7,69054.7,69036.6,69036.8,33.272
2024-05-25 14:36:00,69036.8,69036.8,69013.4,69013.4,25.786


In [67]:
# Set leverage for the needed symbol. You need this because different symbols can have different leverage
def set_leverage(symbol, level):
    try:
        response = client.change_leverage(
            symbol=symbol, leverage=level, recvWindow=6000
        )
        print(response)
    except ClientError as error:
        print(
            "Found error. status: {}, error code: {}, error message: {}".format(
                error.status_code, error.error_code, error.error_message
            )
        )

# The same for the margin type
def set_mode(symbol, type):
    try:
        response = client.change_margin_type(
            symbol=symbol, marginType=type, recvWindow=6000
        )
        print(response)
    except ClientError as error:
        print(
            "Found error. status: {}, error code: {}, error message: {}".format(
                error.status_code, error.error_code, error.error_message
            )
        )


# Price precision. BTC has 1, XRP has 4
def get_price_precision(symbol):
    resp = client.exchange_info()['symbols']
    for elem in resp:
        if elem['symbol'] == symbol:
            return elem['pricePrecision']


# Amount precision. BTC has 3, XRP has 1
def get_qty_precision(symbol):
    resp = client.exchange_info()['symbols']
    for elem in resp:
        if elem['symbol'] == symbol:
            return elem['quantityPrecision']

get_qty_precision('BTCUSDT')

3

In [68]:

# function to buy or sell an order
def open_order(symbol, side):
    price = float(client.ticker_price(symbol)['price']) # get current price and make float
    qty_precision = get_qty_precision(symbol)           # get quantity precision
    price_precision = get_price_precision(symbol)       # get price precision
    qty = round(volume/price, qty_precision)            # define how much you want to buy and round according to qty precision
    if side == 'buy':
        try: # try to buy a long contract
            resp1 = client.new_order(symbol=symbol, side='BUY', type='LIMIT', quantity=qty, timeInForce='GTC', price=price)
            print(symbol, side, "placing order")
            print(resp1)
            sleep(2)   # sleep so that order can be placed 
            sl_price = round(price - price*sl, price_precision) # calculate stop loss price and round
            resp2 = client.new_order(symbol=symbol, side='SELL', type='STOP_MARKET', quantity=qty, timeInForce='GTC', stopPrice=sl_price)
            print(resp2) # create stop loss order and print response
            sleep(2)
            tp_price = round(price + price * tp, price_precision) # calc take profit price and create new order
            resp3 = client.new_order(symbol=symbol, side='SELL', type='TAKE_PROFIT_MARKET', quantity=qty, timeInForce='GTC',
                                     stopPrice=tp_price)
            print(resp3)
        except ClientError as error:
            print(
                "Found error. status: {}, error code: {}, error message: {}".format(
                    error.status_code, error.error_code, error.error_message
                )
            )
    # same for sell side just change small details like side type and sl and tp calculations
    if side == 'sell':
        try:
            resp1 = client.new_order(symbol=symbol, side='SELL', type='LIMIT', quantity=qty, timeInForce='GTC', price=price)
            print(symbol, side, "placing order")
            print(resp1)
            sleep(2)
            sl_price = round(price + price*sl, price_precision)
            resp2 = client.new_order(symbol=symbol, side='BUY', type='STOP_MARKET', quantity=qty, timeInForce='GTC', stopPrice=sl_price)
            print(resp2)
            sleep(2)
            tp_price = round(price - price * tp, price_precision)
            resp3 = client.new_order(symbol=symbol, side='BUY', type='TAKE_PROFIT_MARKET', quantity=qty, timeInForce='GTC',
                                     stopPrice=tp_price)
            print(resp3)
        except ClientError as error:
            print(
                "Found error. status: {}, error code: {}, error message: {}".format(
                    error.status_code, error.error_code, error.error_message
                )
            )


In [69]:
# function to list all pairs with open positions
def get_pos():
    try:
        resp = client.get_position_risk()
        pos = []
        for elem in resp:
            if float(elem['positionAmt']) != 0:
                pos.append(elem['symbol'])
        return pos
    except ClientError as error:
        print(
            "Found error. status: {}, error code: {}, error message: {}".format(
                error.status_code, error.error_code, error.error_message
            )
        )
# function to list all pairs with open orders
def check_orders():
    try:
        response = client.get_orders(recvWindow=6000)
        sym = []
        for elem in response:
            sym.append(elem['symbol'])
        return sym
    except ClientError as error:
        print(
            "Found error. status: {}, error code: {}, error message: {}".format(
                error.status_code, error.error_code, error.error_message
            )
        )

# Close open orders for the needed symbol. If one stop order is executed and another one is still there
def close_open_orders(symbol):
    try:
        response = client.cancel_open_orders(symbol=symbol, recvWindow=6000)
        print(response)
    except ClientError as error:
        print(
            "Found error. status: {}, error code: {}, error message: {}".format(
                error.status_code, error.error_code, error.error_message
            )
        )
print(get_pos())
print(check_orders())

['XTZUSDT']
['XTZUSDT']


In [70]:
# Generates trading signals based on MACD crossover and EMA position.

def macd_ema(symbol):
    
    kl = klines(symbol)       # Fetch historical candlestick data (OHLCV) for the symbol
    macd = ta.trend.macd_diff(kl.Close)  # Calculate the MACD histogram (macd - signal line), kl.Close closing price
    ema = ta.trend.ema_indicator(kl.Close, window=200)  # Calculate the 200-day exponential moving average (EMA)

    # Buy signal conditions:
    # 1. MACD line crosses above zero from below (MACD was negative for two previous periods and is now positive)
    # 2. 200-day EMA is below the current closing price (indicating an overall uptrend)
    if macd.iloc[-3] < 0 and macd.iloc[-2] < 0 and macd.iloc[-1] > 0 and ema.iloc[-1] < kl.Close.iloc[-1]:
        return 'up'

    # Sell signal conditions:
    # 1. MACD line crosses below zero from above (MACD was positive for two previous periods and is now negative)
    # 2. 200-day EMA is above the current closing price (indicating an overall downtrend)
    if macd.iloc[-3] > 0 and macd.iloc[-2] > 0 and macd.iloc[-1] < 0 and ema.iloc[-1] > kl.Close.iloc[-1]:
        return 'down'

    # No signal if conditions are not met
    else:
        return 'none'
macd_ema("BTCUSDT")


'none'

In [71]:
# Ichimoku Cloud Breakout Strategy
def ichimoku_signal(symbol):
    
    kl = klines(symbol)
    ichimoku = ta.trend.IchimokuIndicator(
        high=kl.High, low=kl.Low,
        window1=9, window2=26, window3=52
    )
    
    # Directly access values from the last two candles
    close_prev = kl.Close.iloc[-2]
    close_curr = kl.Close.iloc[-1]
    span_a_prev = ichimoku.ichimoku_a().iloc[-2]
    span_a_curr = ichimoku.ichimoku_a().iloc[-1]
    span_b_prev = ichimoku.ichimoku_b().iloc[-2]
    span_b_curr = ichimoku.ichimoku_b().iloc[-1]

    if close_prev < span_a_prev and close_prev < span_b_prev and close_curr > span_a_curr and close_curr > span_b_curr:
        return 'up'  # Buy: Price crossed above both cloud lines
    elif close_prev > span_a_prev and close_prev > span_b_prev and close_curr < span_a_curr and close_curr < span_b_curr:
        return 'down'  # Sell: Price crossed below both cloud lines
    else:
        return 'none'
ichimoku_signal('BTCUSDT')

'none'

In [72]:
# Generates trading signals based on Bollinger Band breakouts.
def bb_signal(symbol, std_dev=2):
   
    kl = klines(symbol)
    bb_indicator = ta.volatility.BollingerBands(kl.Close, window=20, window_dev=std_dev)

    upper_band = bb_indicator.bollinger_hband()
    lower_band = bb_indicator.bollinger_lband()
#BB signals are complex to analyse, this is just one way to look at it 
    # Buy signal conditions:
    # 1. Current closing price crosses above the upper Bollinger Band
    if kl.Close.iloc[-2] < upper_band.iloc[-2] and kl.Close.iloc[-1] > upper_band.iloc[-1]:
        return 'down'

    # Sell signal conditions:
    # 1. Current closing price crosses below the lower Bollinger Band
    if kl.Close.iloc[-2] > lower_band.iloc[-2] and kl.Close.iloc[-1] < lower_band.iloc[-1]:
        return 'up'

    else:
        return 'none'
    
print(bb_signal('BTCUSDT'))

none


In [73]:
def rsi_signal(symbol):
    kl = klines(symbol)
    rsi = ta.momentum.RSIIndicator(kl.Close).rsi()
    ema = ta.trend.ema_indicator(kl.Close, window=200)
    if rsi.iloc[-2] < 30 and rsi.iloc[-1] > 30:
        return 'up'
    if rsi.iloc[-2] > 70 and rsi.iloc[-1] < 70:
        return 'down'

    else:
        return 'none'
rsi_signal('BTCUSDT')

'none'

In [75]:
# Generates trading signals based on a combination of RSI, Bollinger Bands, Ichimoku Cloud, and MACD with EMA.

def combined_signal(symbol):

    kl = klines(symbol)

    # Get individual signals
    rsi_sig = rsi_signal(symbol)
    bb_sig = bb_signal(symbol)
    ichimoku_sig = ichimoku_signal(symbol)
    macd_ema_sig = macd_ema(symbol)

    # Prioritize signals 
    if rsi_sig == 'up' and bb_sig == 'up' and ichimoku_sig == 'up':
        return 'strong_buy'  # All three indicators agree on a strong buy
    elif rsi_sig == 'down' and bb_sig == 'down' and ichimoku_sig == 'down':
        return 'strong_sell'  # All three indicators agree on a strong sell

    # Handle other combinations of signals 
    elif rsi_sig == bb_sig and rsi_sig != 'none':  # RSI and BB agree, prioritize them
        return rsi_sig
    elif ichimoku_sig != 'none':
        return ichimoku_sig
    elif macd_ema_sig != 'none':
        return macd_ema_sig
    else:
        return 'none'  # No clear signal
    
symbol='BTCUSDT'
combined_signal(symbol)

'none'

In [77]:
orders = 0
symbol = ''
# getting all symbols from Binace Futures list:
symbols = get_tickers_usdt()

while True:
    # we need to get balance to check if the connection is good, or you have all the needed permissions
    balance = get_balance_usdt()
    sleep(1)
    if balance == None:
        print('Cant connect to API. Check IP, restrictions or wait some time')
    if balance != None:
        print("My balance is: ", balance, " USDT")
        # getting position list:
        pos = []
        pos = get_pos()
        print(f'You have {len(pos)} opened positions:\n{pos}')
        # Getting order list
        ord = []
        ord = check_orders()
        # removing stop orders for closed positions
        for elem in ord:
            if not elem in pos:  # check if any order is without an open position
                close_open_orders(elem)  # if yes close it
        # check if you can open new positions according to desired order quantity
        if len(pos) < qty:
            for elem in symbols:

                #signal = macd_ema(elem) # Using MACD Strategy
                #signal = ichimoku_signal(elem) # Using Ichimoku Cloud Strategy
                #signal = bb_signal(elem) # Using Bollinger Bands Strategy                
                signal = combined_signal(elem)
                
                # 'up' or 'down' signal, we place orders for symbols that arent in the opened positions and orders
                # we also dont need USDTUSDC because its 1:1 (waste because commission)
                if (signal == 'up' or signal == 'strong_buy') and elem != 'USDCUSDT' and not elem in pos and not elem in ord and elem != symbol:
                    print('Found BUY signal for ', elem)
                    set_mode(elem, type)
                    sleep(1)
                    set_leverage(elem, leverage)
                    sleep(1)
                    print('Placing order for ', elem)
                    open_order(elem, 'buy')
                    symbol = elem
                    order = True
                    pos = get_pos()   # update the pos list
                    sleep(1)
                    ord = check_orders() # update the ord list
                    sleep(1)
                    sleep(10)
                    break  # add '#' if you dont want to limit amount of orders
                elif (signal == 'down' or signal == 'strong_sell') and elem != 'USDCUSDT' and not elem in pos and not elem in ord and elem != symbol:
                    print('Found SELL signal for ', elem)
                    set_mode(elem, type)
                    sleep(1)
                    set_leverage(elem, leverage)
                    sleep(1)
                    print('Placing order for ', elem)
                    open_order(elem, 'sell')
                    symbol = elem
                    order = True
                    pos = get_pos()   # update the pos list
                    sleep(1)
                    ord = check_orders() # update the ord list
                    sleep(1)
                    sleep(10)
                    break  # add '#' if you dont want to limit amount of orders
    print('Waiting 3 min')
    sleep(180)

My balance is:  82.07852952  USDT
You have 2 opened positions:
['SUSHIUSDT', 'XTZUSDT']
{'code': 200, 'msg': 'The operation of cancel all open order is done.'}
Found BUY signal for  EOSUSDT
{'code': 200, 'msg': 'success'}
{'symbol': 'EOSUSDT', 'leverage': 10, 'maxNotionalValue': '1000000'}
Placing order for  EOSUSDT
EOSUSDT buy placing order
{'orderId': 36261585666, 'symbol': 'EOSUSDT', 'status': 'NEW', 'clientOrderId': 'kdQmH9PiWd4N8iLGCJpZij', 'price': '0.850', 'avgPrice': '0.00', 'origQty': '11.8', 'executedQty': '0.0', 'cumQty': '0.0', 'cumQuote': '0.0000', 'timeInForce': 'GTC', 'type': 'LIMIT', 'reduceOnly': False, 'closePosition': False, 'side': 'BUY', 'positionSide': 'BOTH', 'stopPrice': '0.000', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'LIMIT', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'NONE', 'goodTillDate': 0, 'updateTime': 1716649596527}
{'orderId': 36261585992, 'symbol': 'EOSUSDT', 'status': 'NEW', 'clientOrderId': 'P2t81Kk7jWf5r24DffZa8k',

KeyboardInterrupt: 